## 📥 1. Importación y Carga de los 4 Dataframes Limpios

Cargaremos los 4 dataframes desde los notebooks ya ejecutados. Esto requiere que los notebooks anteriores hayan sido ejecutados y los dataframes estén disponibles.

In [ ]:
# Importaciones necesarias
import pandas as pd
import numpy as np
from pathlib import Path
import os

# Mostrar opciones de pandas para debugging
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

print('Librerías cargadas correctamente.')

### Verificación de disponibilidad de dataframes

In [ ]:
# Función para verificar y reportar disponibilidad de dataframes
def check_dataframes():
    dfs_needed = {
        'df_clientes_True': 'Clientes',
        'df_Ventas_True': 'Ventas (cabecera)',
        'df_detalle_ventas_True': 'Detalle de ventas',
        'df_productos_True': 'Productos'
    }
    
    available = {}
    missing = []
    
    for var_name, label in dfs_needed.items():
        if var_name in globals():
            df_temp = globals()[var_name]
            available[var_name] = label
            print(f'✅ {label:30s} -> Dimensiones: {df_temp.shape}')
        else:
            missing.append(f'{label} ({var_name})')
            print(f'❌ {label:30s} -> NO DISPONIBLE')
    
    if missing:
        print(f'\n⚠️  Falta{"n" if len(missing) > 1 else ""} cargar: {', '.join(missing)}')
        print('\n📌 Por favor, ejecuta primero los notebooks de limpieza:')
        print('   - limp_y_trans_clientes.ipynb')
        print('   - limp_y_trans_ventas.ipynb')
        print('   - limp_y_trans_detalle_ventas.ipynb')
        print('   - limp_y_trans_productos.ipynb')
        return False
    else:
        print(f'\n✨ Todos los dataframes necesarios están disponibles.')
        return True

# Verificar disponibilidad
all_available = check_dataframes()

## 📊 2. Inspección de Estructura y Claves

Antes de realizar los merges, inspeccionaremos cada dataframe para identificar correctamente las claves primarias (PK) y foráneas (FK).

In [ ]:
if all_available:
    print('='*100)
    print('📋 ESTRUCTURA DE DATAFRAMES')
    print('='*100)
    
    print('\n1. df_clientes_True (PK: id_cliente)')
    print('-'*80)
    print(df_clientes_True.head(0))
    print(f'Columnas: {list(df_clientes_True.columns)}')
    print(f'Registros: {len(df_clientes_True)}')
    
    print('\n2. df_Ventas_True (PK: id_venta | FK: id_cliente)')
    print('-'*80)
    print(df_Ventas_True.head(0))
    print(f'Columnas: {list(df_Ventas_True.columns)}')
    print(f'Registros: {len(df_Ventas_True)}')
    
    print('\n3. df_detalle_ventas_True (PK: id_detalle | FK: id_venta, id_producto)')
    print('-'*80)
    print(df_detalle_ventas_True.head(0))
    print(f'Columnas: {list(df_detalle_ventas_True.columns)}')
    print(f'Registros: {len(df_detalle_ventas_True)}')
    
    print('\n4. df_productos_True (PK: id_producto)')
    print('-'*80)
    print(df_productos_True.head(0))
    print(f'Columnas: {list(df_productos_True.columns)}')
    print(f'Registros: {len(df_productos_True)}')

## 🔗 3. Preparación: Renombrado de Claves Foráneas

En la tabla de detalle de ventas, si existe un nombre diferente para la FK de producto (ej: FK_producto), lo renombramos a `id_producto` para que el merge sea unívoco.

In [ ]:
if all_available:
    print('='*100)
    print('🔧 PREPARACIÓN: Validación y renombrado de columnas')
    print('='*100)
    
    # Verificar y renombrar columna de producto en detalle_ventas si es necesario
    if 'FK_producto' in df_detalle_ventas_True.columns:
        print('\n✓ Renombrando FK_producto → id_producto en df_detalle_ventas_True')
        df_detalle_ventas_True.rename(columns={'FK_producto': 'id_producto'}, inplace=True)
    elif 'id_producto' not in df_detalle_ventas_True.columns:
        print('\n⚠️  Aviso: No se encontró FK_producto ni id_producto en detalle_ventas')
        print(f'   Columnas disponibles: {list(df_detalle_ventas_True.columns)}')
    else:
        print('\n✓ Columna id_producto ya existe en df_detalle_ventas_True')
    
    # Validar que todos los FK referencias existan en las PK correspondientes
    print('\n' + '-'*80)
    print('📌 Validación de Integridad Referencial:')
    print('-'*80)
    
    # FK id_cliente en ventas
    clientes_en_ventas = set(df_Ventas_True['id_cliente'].dropna().unique())
    clientes_existentes = set(df_clientes_True['id_cliente'].unique())
    clientes_huerfanos = clientes_en_ventas - clientes_existentes
    print(f'\n✓ FK id_cliente en ventas:',)
    if not clientes_huerfanos:
        print(f'OK ({len(clientes_en_ventas)} referencias válidas)')
    else:
        print(f'INCONSISTENCIA: {len(clientes_huerfanos)} huérfanos')
    
    # FK id_venta en detalle
    ventas_en_detalle = set(df_detalle_ventas_True['id_venta'].dropna().unique())
    ventas_existentes = set(df_Ventas_True['id_venta'].unique())
    ventas_huerfanas = ventas_en_detalle - ventas_existentes
    print(f'✓ FK id_venta en detalle:',)
    if not ventas_huerfanas:
        print(f'OK ({len(ventas_en_detalle)} referencias válidas)')
    else:
        print(f'INCONSISTENCIA: {len(ventas_huerfanas)} huérfanos')
    
    # FK id_producto en detalle
    productos_en_detalle = set(df_detalle_ventas_True['id_producto'].dropna().unique())
    productos_existentes = set(df_productos_True['id_producto'].unique())
    productos_huerfanos = productos_en_detalle - productos_existentes
    print(f'✓ FK id_producto en detalle:',)
    if not productos_huerfanos:
        print(f'OK ({len(productos_en_detalle)} referencias válidas)')
    else:
        print(f'INCONSISTENCIA: {len(productos_huerfanos)} huérfanos')

## 🔀 4. Merge de Tablas - Consolidación de Base Final

Realizamos los merges en orden siguiendo la estructura de relaciones:
1. `df_clientes_True` ← merge ← `df_Ventas_True` (on=id_cliente)
2. Resultado ← merge ← `df_detalle_ventas_True` (on=id_venta)
3. Resultado ← merge ← `df_productos_True` (on=id_producto)

In [ ]:
if all_available:
    print('='*100)
    print('🔀 MERGE - Consolidación de Base Final')
    print('='*100)
    
    print('\n1️⃣  Merge: df_clientes_True + df_Ventas_True (on=id_cliente)')
    print('-'*80)
    base_consolidada = df_clientes_True.merge(
        df_Ventas_True,
        on='id_cliente',
        how='inner',  # inner join para evitar clientes sin ventas
        validate='1:m'  # 1 cliente : muchas ventas
    )
    print(f'✓ Dimensiones resultado: {base_consolidada.shape}')
    print(f'  - Registros: {len(base_consolidada)}')
    print(f'  - Columnas: {len(base_consolidada.columns)}')
    
    print('\n2️⃣  Merge: resultado + df_detalle_ventas_True (on=id_venta)')
    print('-'*80)
    base_consolidada = base_consolidada.merge(
        df_detalle_ventas_True,
        on='id_venta',
        how='inner',  # inner join para que todo tenga detalle
        validate='m:m'  # muchas ventas : muchos detalles
    )
    print(f'✓ Dimensiones resultado: {base_consolidada.shape}')
    print(f'  - Registros: {len(base_consolidada)}')
    print(f'  - Columnas: {len(base_consolidada.columns)}')
    
    print('\n3️⃣  Merge: resultado + df_productos_True (on=id_producto)')
    print('-'*80)
    base_consolidada = base_consolidada.merge(
        df_productos_True,
        on='id_producto',
        how='inner',  # inner join para evitar productos sin venta
        validate='m:1'  # muchos detalles : 1 producto
    )
    print(f'✓ Dimensiones resultado: {base_consolidada.shape}')
    print(f'  - Registros: {len(base_consolidada)}')
    print(f'  - Columnas: {len(base_consolidada.columns)}')
    
    print('\n✨ Merge completado exitosamente!')

## 📊 5. Inspección de la Base Consolidada

In [ ]:
if 'base_consolidada' in locals():
    print('='*100)
    print('📋 INSPECCIÓN - Base Consolidada')
    print('='*100)
    
    print('\nPrimeras 5 filas:')
    print('-'*80)
    print(base_consolidada.head())
    
    print('\n\nÚltimas 5 filas:')
    print('-'*80)
    print(base_consolidada.tail())
    
    print('\n\nTipos de datos:')
    print('-'*80)
    print(base_consolidada.dtypes)
    
    print('\n\nValores nulos por columna:')
    print('-'*80)
    nulls = base_consolidada.isnull().sum()
    if nulls.sum() == 0:
        print('✓ NO hay valores nulos en la base consolidada')
    else:
        print(nulls[nulls > 0])
    
    print('\n\nEstadísticas generales:')
    print('-'*80)
    print(f'Dimensiones: {base_consolidada.shape}')
    print(f'Memoria usada: {base_consolidada.memory_usage(deep=True).sum() / (1024**2):.2f} MB')
    print(f'Clientes únicos: {base_consolidada["id_cliente"].nunique()}')
    print(f'Ventas (id_venta) únicas: {base_consolidada["id_venta"].nunique()}')
    print(f'Detalles de venta únicos: {base_consolidada["id_detalle"].nunique() if "id_detalle" in base_consolidada.columns else "N/A"}')
    print(f'Productos únicos: {base_consolidada["id_producto"].nunique()}')

## 🧹 6. Preprocesamiento y Limpieza Adicional

En esta sección aplicamos transformaciones finales para garantizar que la base esté lista para análisis.

In [ ]:
if 'base_consolidada' in locals():
    print('='*100)
    print('🧹 PREPROCESAMIENTO - Limpieza y Transformaciones Finales')
    print('='*100)
    
    # Crear copia para no modificar original hasta validar
    base_final = base_consolidada.copy()
    
    # 1. Detectar y manejar duplicados
    print('\n1️⃣  Detección de duplicados')
    print('-'*80)
    dup_count = base_final.duplicated().sum()
    print(f'Filas duplicadas: {dup_count}')
    if dup_count > 0:
        print(f'  (Eliminando {dup_count} filas duplicadas)')
        base_final = base_final.drop_duplicates()
        print(f'  Nuevo tamaño: {base_final.shape}')
    
    # 2. Renombrar columnas para mayor claridad y evitar colisiones
    print('\n2️⃣  Estandarización de nombres de columnas')
    print('-'*80)
    rename_map = {}
    for col in base_final.columns:
        # Aquí puedes definir transformaciones de nombres si es necesario
        # Ejemplo: renombrar columnas que tengan sufijos _x o _y
        if col.endswith('_x'):
            rename_map[col] = col[:-2]
        elif col.endswith('_y'):
            rename_map[col] = col[:-2]
    
    if rename_map:
        print(f'Columnas renombradas:')
        for old, new in rename_map.items():
            print(f'  {old:30s} → {new}')
        base_final.rename(columns=rename_map, inplace=True)
    else:
        print('✓ No hay columnas con sufijos _x o _y')
    
    # 3. Reordenar columnas de forma lógica (primero IDs, luego datos descriptivos)
    print('\n3️⃣  Reorganización de columnas (orden lógico)')
    print('-'*80)
    id_cols = [c for c in base_final.columns if 'id' in c.lower()]
    date_cols = [c for c in base_final.columns if 'fecha' in c.lower() or 'date' in c.lower()]
    money_cols = [c for c in base_final.columns if any(x in c.lower() for x in ['precio', 'importe', 'monto', 'total'])]
    other_cols = [c for c in base_final.columns if c not in id_cols + date_cols + money_cols]
    
    new_order = id_cols + date_cols + other_cols + money_cols
    base_final = base_final[[c for c in new_order if c in base_final.columns]]
    print(f'✓ Columnas reordenadas: {len(new_order)} columnas')
    print(f'  - IDs (PK/FK): {len(id_cols)}')
    print(f'  - Fechas: {len(date_cols)}')
    print(f'  - Dinero (precios/importes): {len(money_cols)}')
    print(f'  - Otros: {len(other_cols)}')
    
    # 4. Tipos de datos finales
    print('\n4️⃣  Validación de tipos de datos')
    print('-'*80)
    print(base_final.dtypes)
    
    print('\n✨ Preprocesamiento completado')

## 📄 7. Exportación a CSV

In [ ]:
if 'base_final' in locals():
    print('='*100)
    print('💾 EXPORTACIÓN - Base Final a CSV')
    print('='*100)
    
    # Definir ruta de exportación
    db_path = Path('db')
    db_path.mkdir(exist_ok=True)
    
    export_file = db_path / 'Base_Final_Aurelion.csv'
    
    # Exportar a CSV
    print(f'\n🔄 Exportando a: {export_file}')
    base_final.to_csv(export_file, index=False, encoding='utf-8-sig')
    
    # Validar exportación
    if export_file.exists():
        file_size_mb = export_file.stat().st_size / (1024**2)
        print(f'✅ Exportación exitosa')
        print(f'\n📊 Detalles del archivo:')
        print(f'   - Nombre: {export_file.name}')
        print(f'   - Ruta: {export_file.resolve()}')
        print(f'   - Tamaño: {file_size_mb:.2f} MB')
        print(f'   - Registros: {len(base_final):,}')
        print(f'   - Columnas: {len(base_final.columns)}')
    else:
        print('❌ Error: El archivo no se creó correctamente')
    
    print('\n✨ Consolidación completada exitosamente')

## 📋 8. Resumen Final

In [ ]:
print('='*100)
print('📋 RESUMEN FINAL - CONSOLIDACIÓN PROYECTO AURELION')
print('='*100)

print('\n🎯 Objetivo cumplido:')
print('   ✓ Se integraron los 4 dataframes limpios (clientes, ventas, detalle_ventas, productos)')
print('   ✓ Se aplicaron relaciones de clave primaria (PK) y foránea (FK)')
print('   ✓ Se validó integridad referencial')
print('   ✓ Se realizó preprocesamiento y limpieza adicional')
print('   ✓ Se exportó base consolidada a CSV')

print('\n📊 Estadísticas finales:')
if 'base_final' in locals():
    print(f'   - Registros en base final: {len(base_final):,}')
    print(f'   - Columnas integradas: {len(base_final.columns)}')
    print(f'   - Memoria usada: {base_final.memory_usage(deep=True).sum() / (1024**2):.2f} MB')
    print(f'   - Ubicación archivo CSV: db/Base_Final_Aurelion.csv')

print('\n🚀 Próximos pasos:')
print('   1. Usar Base_Final_Aurelion.csv para análisis avanzados')
print('   2. Crear visualizaciones estratégicas')
print('   3. Desarrollar modelos predictivos o de segmentación')
print('   4. Generar reportes ejecutivos')

print('\n✨ Consolidación finalizada correctamente')